In [4]:
# debug_and_fix_tags.py
import zmq, msgpack, msgpack_numpy as m
m.patch()
import numpy as np, time, math

PORT = 5557
TAG4_ID = 4
TAG4_HEIGHT_M = 0.075  # same as your system
UP = np.array([0.0, 0.0, 1.0])

ctx = zmq.Context()
sock = ctx.socket(zmq.SUB)
sock.setsockopt(zmq.SUBSCRIBE, b"")
sock.connect(f"tcp://127.0.0.1:{PORT}")

def rot_err(R):
    I = np.eye(3)
    return np.linalg.norm(R.dot(R.T) - I), np.linalg.det(R)

def vec_info(name, v):
    return f"{name}: [{v[0]:+.3f}, {v[1]:+.3f}, {v[2]:+.3f}] | |v|={np.linalg.norm(v):.3f}"

def align_local_z_to_world_up(T):
    """
    Returns T_fixed = T.dot(R_local) where R_local rotates tag-local axes
    so that tag-local +Z (0,0,1) maps to world UP.
    This preserves tag position but changes its orientation so its Z aligns to UP.
    """
    R = T[:3,:3]
    n = R[:,2]  # current tag Z in world
    n = n / np.linalg.norm(n)
    # target k in tag-local coordinates: we want R * (R_local * e_z) = UP
    # So R_local * e_z = R.T @ UP = k_local
    k_local = R.T.dot(UP)
    k_local = k_local / np.linalg.norm(k_local)
    # now compute rotation that maps e_z -> k_local in local coordinates
    ez = np.array([0.0, 0.0, 1.0])
    v = np.cross(ez, k_local)
    s = np.linalg.norm(v)
    c = np.dot(ez, k_local)
    if s < 1e-8:
        # already aligned or opposite
        if c > 0:
            R_local = np.eye(3)
        else:
            # 180 deg rotation about any orth axis (choose x)
            R_local = np.array([[1,0,0],[0,-1,0],[0,0,-1]], float)
    else:
        vx = np.array([[0, -v[2], v[1]],[v[2], 0, -v[0]],[-v[1], v[0], 0]], float)
        R_local = np.eye(3) + vx + vx.dot(vx) * ((1 - c) / (s**2))
    T_fixed = np.eye(4)
    T_fixed[:3,:3] = R.dot(R_local)  # new world rotation
    T_fixed[:3,3] = T[:3,3].copy()
    return T_fixed

print("[debug] waiting for snapshot...")
msg = sock.recv()   # blocking for one snapshot
data = msgpack.unpackb(msg, object_hook=m.decode, strict_map_key=False)

tags = data.get("tags", {})
cams = data.get("cameras", {})

print("--- SNAPSHOT CHECK ---\n")
for tid, T in tags.items():
    T = np.array(T, dtype=float)
    pos = T[:3,3]
    R = T[:3,:3]
    err, det = rot_err(R)
    n = R[:,2]  # tag local +Z in world
    print(f"TAG {tid}")
    print(f"  pos = {pos}")
    print(f"  normal (world) = {n}  | dot(up) = {n.dot(UP):.3f}")
    print(f"  rot ortho err = {err:.6f}, det = {det:.6f}")
    # check if tag is floor-like (z small) or mobile
    z = pos[2]
    print(f"  z = {z:.4f} m")
    # decide if orientation is bad for a floor tag
    if tid != TAG4_ID:
        if abs(z) < 0.5:
            # expected floor tag -> normal dot up should be near ±1
            if abs(n.dot(UP)) < 0.8:
                print("  >>> WARNING: tag normal not aligned with UP (likely tilt/vertical).")
                T_fix = align_local_z_to_world_up(T)
                nfix = T_fix[:3,:3][:,2]
                print("    -> Suggest fixed normal (world) = ", nfix, "pos unchanged:", T_fix[:3,3])
            else:
                print("  orientation looks OK for floor tag.")
        else:
            print("  tag has high z; may be on raised surface (bed/shelf).")
    else:
        # mobile tag: check height near TAG4_HEIGHT_M
        if abs(z - TAG4_HEIGHT_M) > 0.05:
            print(f"  >>> WARNING: mobile tag z != {TAG4_HEIGHT_M:.3f} m (delta {z-TAG4_HEIGHT_M:+.3f})")
            # give fix that sets z to expected height but keeps orientation
            T_fixed = T.copy()
            T_fixed[2,3] = TAG4_HEIGHT_M
            print("    -> suggested T with corrected height (translation):", T_fixed[:3,3])
        # mobile orientation check (often vertical or horizontal depending on mount)
        print("  mobile tag normal dot(up) = ", n.dot(UP))

print("\nCameras:")
for cname, T in cams.items():
    T = np.array(T, dtype=float)
    pos = T[:3,3]
    R = T[:3,:3]
    err, det = rot_err(R)
    print(f"CAM {cname}: pos={pos}, rot_err={err:.6f}, det={det:.6f}")

print("\n--- DONE ---")


[debug] waiting for snapshot...
--- SNAPSHOT CHECK ---

TAG 1
  pos = [0. 0. 0.]
  normal (world) = [0. 0. 1.]  | dot(up) = 1.000
  rot ortho err = 0.000000, det = 1.000000
  z = 0.0000 m
  orientation looks OK for floor tag.
TAG 5
  pos = [ 0.29381215  0.26504174 -0.0290148 ]
  normal (world) = [-0.01339847  0.09582793  0.99530774]  | dot(up) = 0.995
  rot ortho err = 0.000000, det = 1.000000
  z = -0.0290 m
  orientation looks OK for floor tag.
TAG 4
  pos = [0.28268027 0.45983725 0.075     ]
  normal (world) = [0.04970696 0.01694327 0.99862012]  | dot(up) = 0.999
  rot ortho err = 0.000000, det = 1.000000
  z = 0.0750 m
  mobile tag normal dot(up) =  0.9986201199287998
TAG 0
  pos = [ 0.84994872 -0.05455263  0.04684224]
  normal (world) = [ 0.01719388 -0.00991732  0.99980299]  | dot(up) = 1.000
  rot ortho err = 0.000000, det = 1.000000
  z = 0.0468 m
  orientation looks OK for floor tag.

Cameras:
CAM kreo1: pos=[ 0.21205152 -1.34469178  2.0195015 ], rot_err=0.000000, det=1.000000


In [16]:
import open3d as o3d
import numpy as np
import pandas as pd
import time
import sys
import os
import glob

# ---------- Configuration ----------
BALL_RADIUS = 0.025
CAM1_POS = [0.15, -1.4, 2.0]
CAM2_POS = [1.1, 1.8, 2.0]
LOOP_DELAY = 5.0
MIN_MOVE_DIST = 0.0001 

# ---------- Helper Functions ----------
def create_custom_grid(size=4.0, step=0.3):
    lines = []
    points = []
    start = -size
    end = size
    num_steps = int((end - start) / step) + 1
    
    for i in range(num_steps):
        y = start + i * step
        points.append([-size, y, 0]); points.append([size, y, 0])
        lines.append([len(points)-2, len(points)-1])
        
        x = start + i * step
        points.append([x, -size, 0]); points.append([x, size, 0])
        lines.append([len(points)-2, len(points)-1])

    colors = [[0.3, 0.3, 0.3] for _ in range(len(lines))]
    grid = o3d.geometry.LineSet()
    grid.points = o3d.utility.Vector3dVector(points)
    grid.lines = o3d.utility.Vector2iVector(lines)
    grid.colors = o3d.utility.Vector3dVector(colors)
    return grid

def create_robot_mesh():
    robot_box = o3d.geometry.TriangleMesh.create_box(0.25, 0.25, 0.075)
    robot_box.translate([-0.2, -0.15, 0])
    robot_box.paint_uniform_color([0.1, 0.1, 0.9]) 
    dustbin = o3d.geometry.TriangleMesh.create_cylinder(radius=0.06, height=0.12)
    dustbin.translate([-0.05, 0, 0.075])
    dustbin.paint_uniform_color([0.9, 0.2, 0.2]) 
    robot_mesh = robot_box + dustbin
    robot_mesh.compute_vertex_normals()
    return robot_mesh

def compute_robot_center(row):
    # Extract Tag 4
    p4 = None
    if not pd.isna(row['tag4_x']) and str(row['tag4_x']) != '':
        p4 = np.array([float(row['tag4_x']), float(row['tag4_y']), float(row['tag4_z'])])

    # Extract Tag 5
    p5 = None
    if not pd.isna(row['tag5_x']) and str(row['tag5_x']) != '':
        p5 = np.array([float(row['tag5_x']), float(row['tag5_y']), float(row['tag5_z'])])

    shift_m = 0.096
    default_y_axis = np.array([0.0, 1.0, 0.0])

    if p4 is not None and p5 is not None:
        return 0.5 * (p4 + p5)
    if p4 is not None:
        return p4 + (-shift_m) * default_y_axis
    if p5 is not None:
        return p5 + (shift_m) * default_y_axis
    return None

# ---------- Main App Logic ----------
class LogVisualizerLegacy:
    def __init__(self):
        # Data Loading
        self.data = self.find_and_load_csv()
        self.frame_idx = 0
        self.fps = 30
        self.is_paused = False
        self.ball_path_points = []
        
        # Geometry Placeholders
        self.ball_geom = o3d.geometry.TriangleMesh.create_sphere(radius=BALL_RADIUS)
        self.ball_geom.paint_uniform_color([1.0, 0.5, 0.0])
        self.ball_geom.compute_vertex_normals()
        
        self.robot_geom = create_robot_mesh()
        
        # CHANGED: Use PointCloud instead of LineSet
        self.trail_geom = o3d.geometry.PointCloud()

        # Initialize Visualizer
        self.vis = o3d.visualization.VisualizerWithKeyCallback()
        self.vis.create_window(window_name="Ball Catcher Replay (Legacy)", width=1280, height=720)

        # Increase point size so the trail is visible
        opt = self.vis.get_render_option()
        opt.point_size = 5.0
        opt.background_color = np.array([0.1, 0.1, 0.1])

        # Add Geometries
        self.vis.add_geometry(self.ball_geom)
        self.vis.add_geometry(self.robot_geom)
        self.vis.add_geometry(create_custom_grid())
        self.vis.add_geometry(o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5))
        
        # Add Cameras
        for i, pos in enumerate([CAM1_POS, CAM2_POS]):
            cam = o3d.geometry.TriangleMesh.create_cone(radius=0.1, height=0.2)
            cam.paint_uniform_color([0,1,0] if i==1 else [1,0,0])
            cam.compute_vertex_normals()
            t = np.eye(4); t[:3,3] = pos
            # Rotate cone to point down
            R = cam.get_rotation_matrix_from_xyz((np.pi,0,0))
            cam.rotate(R, center=(0,0,0))
            cam.transform(t)
            self.vis.add_geometry(cam)

        # View Control
        ctr = self.vis.get_view_control()
        ctr.set_lookat([0, 0, 0])
        ctr.set_front([0, -1, 1]) # Look from y=-1, z=1 direction
        ctr.set_up([0, 0, 1])
        ctr.set_zoom(0.8)

        # Register Callbacks
        self.vis.register_key_callback(ord(' '), self.toggle_pause)
        self.vis.register_key_callback(ord('+'), self.increase_fps)
        self.vis.register_key_callback(ord('='), self.increase_fps) # For keyboards where + is =
        self.vis.register_key_callback(ord('-'), self.decrease_fps)
        self.vis.register_animation_callback(self.update_frame)

    def find_and_load_csv(self):
        files = glob.glob("../detection/data_log_1763477064.csv")
        if not files: print("No CSV found!"); sys.exit()
        latest = max(files, key=os.path.getmtime)
        print(f"Loading: {latest}")
        return pd.read_csv(latest)

    def toggle_pause(self, vis):
        self.is_paused = not self.is_paused
        print(f"Paused: {self.is_paused}")
        return False

    def increase_fps(self, vis):
        self.fps = min(65, self.fps + 5)
        print(f"FPS: {self.fps}")
        return False

    def decrease_fps(self, vis):
        self.fps = max(5, self.fps - 5)
        print(f"FPS: {self.fps}")
        return False

    def update_frame(self, vis):
        if self.is_paused: return False
        
        # Simple Frame Throttling
        time.sleep(1.0 / self.fps)

        if self.frame_idx >= len(self.data):
            print(f"Looping in {LOOP_DELAY}s...")
            time.sleep(LOOP_DELAY)
            self.frame_idx = 0
            self.ball_path_points = []
            
            # Clear trail visual (Empty PointCloud)
            self.trail_geom.points = o3d.utility.Vector3dVector([])
            vis.update_geometry(self.trail_geom)
            return True

        row = self.data.iloc[self.frame_idx]
        
        # 1. Update Ball
        bx, by, bz = row['ball_x'], row['ball_y'], row['ball_z']
        if not pd.isna(bx) and str(bx) != '':
            pos = np.array([float(bx), float(by), float(bz)])
            # Translate relative to current center (reset to 0 then move)
            center = self.ball_geom.get_center()
            self.ball_geom.translate(pos - center)
            vis.update_geometry(self.ball_geom)
            
            # Update Trail
            self.update_trail(vis, pos)
        else:
            # Move away
            center = self.ball_geom.get_center()
            self.ball_geom.translate(np.array([0,0,-100]) - center)
            vis.update_geometry(self.ball_geom)

        # 2. Update Robot
        rpos = compute_robot_center(row)
        if rpos is not None:
            center = self.robot_geom.get_center()
            self.robot_geom.translate(rpos - center)
            vis.update_geometry(self.robot_geom)
        
        self.frame_idx += 1
        return True

    def update_trail(self, vis, new_pt):
        # Add point logic
        if len(self.ball_path_points) > 0:
            dist = np.linalg.norm(new_pt - self.ball_path_points[-1])
            if dist < MIN_MOVE_DIST: return

        self.ball_path_points.append(new_pt)
        
        # Update PointCloud Geometry
        points = np.array(self.ball_path_points)
        self.trail_geom.points = o3d.utility.Vector3dVector(points)
        self.trail_geom.paint_uniform_color([0, 1, 0]) # Green
        
        # If it's the first time adding points, we might need to add geometry
        if len(self.ball_path_points) == 1:
             vis.add_geometry(self.trail_geom, reset_bounding_box=False)
        else:
             vis.update_geometry(self.trail_geom)

    def run(self):
        self.vis.run()
        self.vis.destroy_window()

if __name__ == "__main__":
    app = LogVisualizerLegacy()
    app.run()

Loading: ../detection/data_log_1763477064.csv
Looping in 5.0s...


In [2]:
import open3d as o3d
import open3d.visualization.gui as gui
import open3d.visualization.rendering as rendering
import numpy as np
import pandas as pd
import time
import sys
import os
import glob

# ---------- Configuration ----------
BALL_RADIUS = 0.025  # 2.5 cm
ROBOT_DIMS = [0.3, 0.4, 0.2]  # Width, Length, Height (approx)
CAM1_POS = [0.15, -1.4, 2.0]
CAM2_POS = [1.1, 1.8, 2.0]
LOOP_DELAY = 5.0  # Seconds

# ---------- Robot Logic (Adapted) ----------
def compute_robot_center(row):
    """
    Reconstructs tag info from CSV row and applies user logic.
    Note: CSV lacks orientation (y_axis_unit), so we default to [0,1,0] for single tags.
    """
    # Extract Tag 4
    p4 = None
    if not pd.isna(row['tag4_x']) and str(row['tag4_x']) != '':
        p4 = np.array([float(row['tag4_x']), float(row['tag4_y']), float(row['tag4_z'])])

    # Extract Tag 5
    p5 = None
    if not pd.isna(row['tag5_x']) and str(row['tag5_x']) != '':
        p5 = np.array([float(row['tag5_x']), float(row['tag5_y']), float(row['tag5_z'])])

    shift_m = 0.096
    # Default assumption for Y-axis since not in CSV
    default_y_axis = np.array([0.0, 1.0, 0.0])

    if p4 is not None and p5 is not None:
        center = 0.5 * (p4 + p5)
        return center
    if p4 is not None:
        center = p4 + (-shift_m) * default_y_axis
        return center
    if p5 is not None:
        center = p5 + (shift_m) * default_y_axis
        return center
    return None

# ---------- Visualizer Application ----------
class LogVisualizerApp:
    def __init__(self):
        # 1. Initialize Open3D GUI
        self.app = gui.Application.instance
        self.app.initialize()
        
        self.window = self.app.create_window("Log Visualizer", 1280, 720)
        self.widget3d = gui.SceneWidget()
        self.widget3d.scene = rendering.Open3DScene(self.window.renderer)
        self.widget3d.scene.set_background([0.1, 0.1, 0.1, 1.0])  # Dark Gray

        # 2. GUI Panel (FPS Slider)
        self.panel = gui.Vert(0, gui.Margins(10, 10, 10, 10))
        self.panel.preferred_width = 250
        
        lab = gui.Label("Playback FPS")
        self.slider = gui.Slider(gui.Slider.INT)
        self.slider.set_limits(5, 65)
        self.slider.int_value = 30  # Default FPS
        
        self.panel.add_child(lab)
        self.panel.add_child(self.slider)

        # Layout Callback
        self.window.set_on_layout(self._on_layout)
        self.window.add_child(self.widget3d)
        self.window.add_child(self.panel)

        # 3. Load Data
        self.data = self.find_and_load_csv()
        self.frame_idx = 0
        self.is_waiting = False
        self.wait_start_time = 0
        self.last_update_time = time.time()

        # 4. Setup Scene Geometry
        self.setup_scene()

        # 5. Start Loop
        self.window.set_on_tick_event(self._on_tick)

    def find_and_load_csv(self):
        # Find most recent CSV in current directory
        files = glob.glob("../detection/*.csv")
        if not files:
            print("No CSV files found in current directory!")
            sys.exit()
        # Sort by modification time
        latest_file = max(files, key=os.path.getmtime)
        print(f"Loading latest log: {latest_file}")
        try:
            df = pd.read_csv(latest_file)
            print(f"Loaded {len(df)} frames.")
            return df
        except Exception as e:
            print(f"Error reading CSV: {e}")
            sys.exit()

    def _on_layout(self, layout_context):
        r = self.window.content_rect
        self.widget3d.frame = r
        # Panel on the right
        panel_size = self.panel.calc_preferred_size(layout_context, gui.Widget.Constraints())
        self.panel.frame = gui.Rect(r.width - panel_size.width, r.y,
                                    panel_size.width, panel_size.height)

    def setup_scene(self):
        mat = rendering.MaterialRecord()
        mat.base_color = [1.0, 1.0, 1.0, 1.0]
        mat.shader = "defaultLit"

        # --- Cameras ---
        # Cam 1 (Red)
        cam1 = o3d.geometry.TriangleMesh.create_cone(radius=0.1, height=0.2)
        cam1.paint_uniform_color([1.0, 0.0, 0.0])
        cam1.compute_vertex_normals()
        t1 = np.eye(4); t1[:3, 3] = CAM1_POS
        # Rotate to point downwards-ish
        R_cam = cam1.get_rotation_matrix_from_xyz((np.pi, 0, 0))
        cam1.rotate(R_cam, center=(0,0,0))
        cam1.transform(t1)
        self.widget3d.scene.add_geometry("cam1", cam1, mat)

        # Cam 2 (Green)
        cam2 = o3d.geometry.TriangleMesh.create_cone(radius=0.1, height=0.2)
        cam2.paint_uniform_color([0.0, 1.0, 0.0])
        cam2.compute_vertex_normals()
        t2 = np.eye(4); t2[:3, 3] = CAM2_POS
        cam2.rotate(R_cam, center=(0,0,0))
        cam2.transform(t2)
        self.widget3d.scene.add_geometry("cam2", cam2, mat)

        # --- Floor Grid (Reference) ---
        # Simple grid of lines
        grid = o3d.geometry.LineSet.create_camera_visualization(640, 480, np.eye(3), np.eye(4), 1.0)
        # Or just a coordinate frame
        axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
        self.widget3d.scene.add_geometry("axes", axes, mat)

        # --- Ball (Dynamic) ---
        self.ball_geom = o3d.geometry.TriangleMesh.create_sphere(radius=BALL_RADIUS)
        self.ball_geom.paint_uniform_color([1.0, 0.5, 0.0]) # Orange
        self.ball_geom.compute_vertex_normals()
        self.mat_ball = rendering.MaterialRecord()
        self.mat_ball.base_color = [1.0, 0.5, 0.0, 1.0]
        self.mat_ball.shader = "defaultLit"
        self.widget3d.scene.add_geometry("ball", self.ball_geom, self.mat_ball)

        # --- Robot (Dynamic) ---
        self.robot_geom = o3d.geometry.TriangleMesh.create_box(
            width=ROBOT_DIMS[0], height=ROBOT_DIMS[1], depth=ROBOT_DIMS[2]
        )
        # Center the box on its origin
        self.robot_geom.translate([-ROBOT_DIMS[0]/2, -ROBOT_DIMS[1]/2, -ROBOT_DIMS[2]/2])
        self.robot_geom.paint_uniform_color([0.0, 0.4, 0.8]) # Blue
        self.robot_geom.compute_vertex_normals()
        self.mat_robot = rendering.MaterialRecord()
        self.mat_robot.base_color = [0.0, 0.4, 0.8, 1.0]
        self.mat_robot.shader = "defaultLit"
        self.widget3d.scene.add_geometry("robot", self.robot_geom, self.mat_robot)

        # Set initial camera view
        self.widget3d.look_at([0, 0, 0], [0, -3, 3], [0, 0, 1])

    def _on_tick(self):
        now = time.time()
        
        # 1. Handle Restart Delay
        if self.is_waiting:
            if now - self.wait_start_time > LOOP_DELAY:
                print("Restarting playback...")
                self.is_waiting = False
                self.frame_idx = 0
            return True # FIXED: Return boolean

        # 2. Handle FPS Throttling
        target_fps = self.slider.int_value
        if target_fps <= 0: target_fps = 1
        dt = 1.0 / target_fps
        
        if now - self.last_update_time < dt:
            return True # FIXED: Return boolean
        
        self.last_update_time = now

        # 3. End of Data Check
        if self.frame_idx >= len(self.data):
            print(f"Log finished. Waiting {LOOP_DELAY}s...")
            self.is_waiting = True
            self.wait_start_time = now
            return True # FIXED: Return boolean

        # 4. Update Scene
        row = self.data.iloc[self.frame_idx]
        
        # Update Ball Position
        bx, by, bz = row['ball_x'], row['ball_y'], row['ball_z']
        if not pd.isna(bx) and str(bx) != '':
            t = np.eye(4)
            t[:3, 3] = [float(bx), float(by), float(bz)]
            self.widget3d.scene.set_geometry_transform("ball", t)
        else:
            # Hide ball (move far away)
            t = np.eye(4); t[:3, 3] = [0, 0, -100]
            self.widget3d.scene.set_geometry_transform("ball", t)

        # Update Robot Position
        robot_pos = compute_robot_center(row)
        if robot_pos is not None:
            t = np.eye(4)
            t[:3, 3] = robot_pos
            self.widget3d.scene.set_geometry_transform("robot", t)
        else:
            t = np.eye(4); t[:3, 3] = [0, 0, -100]
            self.widget3d.scene.set_geometry_transform("robot", t)

        self.frame_idx += 1
        
        # Force redraw
        self.window.post_redraw()
        return True # FIXED: Return boolean

    def run(self):
        self.app.run()

if __name__ == "__main__":
    app = LogVisualizerApp()
    app.run()

[Open3D INFO] Window window_1 created.
Loading latest log: ../detection/data_log_1763700127.csv
Loaded 12443 frames.


KeyboardInterrupt: 